In [4]:
df

,Unnamed: 0,referral,output
0,0,e-RS Referral \nPatient Demographics \nPatient...,surgery
1,1,"SHEET \nNEO APPT ohfpois 15-15 \nSPOKE TG Pr, ...",referral
2,2,e-RS Referral \nPatient Demographics \nPatient...,surgery
3,3,e-RS Referral \nPatient Demographics \nPatient...,physio
4,4,e-RS Referral \nPatient Demographics \nPatient...,surgery
...,...,...,...
103,103,e-RS Referral \nPatient Demographics \nPatient...,tests
104,104,e-RS Referral \nPatient Demographics \nPatient...,surgery
105,105,e-RS Referral \nPatient Demographics \nPatient...,tests
106,106,e-RS Referral \nPatient Demographics \nPatient...,discharge


In [15]:
help(tokenizer)

Help on BertTokenizerFast in module transformers.models.bert.tokenization_bert_fast object:

class BertTokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  BertTokenizerFast(vocab_file=None, tokenizer_file=None, do_lower_case=True, unk_token='[UNK]', sep_token='[SEP]', pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]', tokenize_chinese_chars=True, strip_accents=None, **kwargs)
 |  
 |  Construct a "fast" BERT tokenizer (backed by HuggingFace's *tokenizers* library). Based on WordPiece.
 |  
 |  This tokenizer inherits from [`PreTrainedTokenizerFast`] which contains most of the main methods. Users should
 |  refer to this superclass for more information regarding those methods.
 |  
 |  Args:
 |      vocab_file (`str`):
 |          File containing the vocabulary.
 |      do_lower_case (`bool`, *optional*, defaults to `True`):
 |          Whether or not to lowercase the input when tokenizing.
 |      unk_token (`str`, *optional*, defaults to `"[UNK]"`):

In [21]:
from transformers import Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset, load_metric
import torch
from transformers import AutoModelForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Load CSV file into a pandas DataFrame
df = pd.read_csv('/home/swleocresearch/Desktop/triage-ai/datasets/triage_dataset.csv')  # replace 'train.csv' with your CSV file name

# Assuming the first column is the text and the second column is the label
texts = df.iloc[:, 1].tolist()  # First column as input texts
labels = LabelEncoder().fit_transform(df.iloc[:, 2].tolist())  # Second column as labels

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Tokenization function
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=6322)

# Tokenize training and validation data
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

# Convert into Dataset objects
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'label': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'label': val_labels
})


/home/swleocresearch/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
len(train_encodings['input_ids'][0])

6322

In [25]:

model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=7)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:

# Load the model with a classification head
# Define the metric
metric = load_metric("accuracy",trust_remote_code=True)

def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save the model after each epoch
    save_total_limit=2  # Only keep the last 2 models saved
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


/home/swleocresearch/.local/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
# Train the model
trainer.train()

# Save the trained model
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/swleocresearch/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/swleocresearch/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/swleocresearch/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/swleocresearch/.local/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py", line 1695, in forward
    outputs = self.bert(
  File "/home/swleocresearch/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/swleocresearch/.local/lib/python3.10/site-packages/torch/nn/mod